# 5.1 层和块
+ 层(layer)：卷积层、线性映射层....
+ 块(block)：由许多层组成。

In [ ]:
# 块结构
import torch
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
    def forward(self,X):
        y=self.hidden(X)
        y=self.out(y)
        return y

# 5.2 参数管理

In [ ]:
#参数访问
import torch
from torch import nn
net=nn.Sequential(
    nn.Linear(4,8),
    nn.ReLU(),
    nn.Linear(8,1)
)
print(net[2].state_dict())
print(net[2].bias) #tensor([0.1111],requires_grad=True)
print(net[2].bias.data) # tensor([0.1629])

In [ ]:
#一次性访问
print(*[(name, param.shape) for name, param in net.named_parameters()])

In [ ]:
#参数初始化
def xavier(m):
    nn.init.xavier_uniform_(m.weight)
net.apply(xavier)

In [ ]:
#参数绑定
#将层提前定义，然后插入
shared=nn.Linear(8,8)
net=nn.Sequential(
    nn.Linear(4,8),
    nn.ReLU(),
    shared,
    nn.ReLU(),
    shared,
    nn.Linear(8,1)
)
# 由于模型参数包含梯度，
# 因此在反向传播期间第二个隐藏层和第三个隐藏层的梯度会加在一起。

# 读写文件
+ 保存和加载张量
  ```
  import torch
  x=torch.arange(4)
  torch.save(x,'x-file')
  x2=torch.load('x-file')
  ```
+ 加载和保存模型参数
  ```
  import torch
  import torch.nn as nn
  class MLP(nn.Module):
      def __init__(self):
          super().__init__()
          self.hidden=nn.Linear(20,256)
          self.out=nn.Linear(256,10)
      def forward(self,X):
          y=self.hidden(X)
          y=self.out(y)
          return y
  net=MLP()
  torch.save(net.state_dict(),'mlp.params')
  clone_mlp=MLP()
  clone.load_state_dict(torch.load('mlp.params'))
  ```

# 5.5 GPU的使用

In [1]:
# 基本操作
import torch
torch.device('cpu') #使用cpu
torch.cuda.device('cuda') # 使用GPU
torch.cuda.device('cuda:1')
#查询GPU数量
torch.cuda.device_count()
# 使用指定型号的GPU
def try_gpu(index=0):
    if torch.cuda.device_count()>=index+1:
        return torch.device(f"cuda{index}")
    return torch.device('cpu')
# 使用所有GPU
def try_all_gpus():
    devices=[torch.device(f"cuda:{i}") for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

In [ ]:
# 使用GPU
## 张量
X=torch.ones(2,3,device=try_all_gpus())
## 模型
net=nn.Sequential(nn.Linear(3,1))
net=net.to(device=try_all_gpus())